# 10 fiese Fallstricke und 3 mal schwarze Python-Magie


Dieses Jupyter-Notebook enthält den Quelltext für Kapitel 18 »10 fiese Fallstricke und 3 mal schwarze Python-Magie« im Buch [Python für Ingenieure für Dummies](https://python-fuer-ingenieure.de/).

In [1]:
# sorgt dafür, dass, wenn die letzte Zeile eine Zuweisung ist, deren Ergebnis auch ausgegeben wird
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

### Globale und lokale Variablen

`begin variablen1`

In [2]:
X = 123 # globale Variable

def func1():
    print(X) # lesender Zugriff auf globale Variable
    

def func2():
    X = 789 # Schreibzugriff -> Name `X` wird lokal
    print(X) # lesender Zugriff auf lokale Variable
    
func1()

123


In [3]:
func2()

789


`end variablen1`

`begin variablen2`

In [4]:
def func3():
    print(X) # lesender Zugriff 
    X = 789 # Schreibzugriff -> Name `X` wird lokal
    print(X) # lesender Zugriff
    
#!func3()
print("UnboundLocalError: local variable 'X' referenced before assignment") #!

UnboundLocalError: local variable 'X' referenced before assignment


`end variablen2`

### Backslashes in Strings

`begin backslash1`

In [5]:
len("\n"), len("\f"), len("\p")

(1, 1, 2)

In [6]:
print("\frac{\pi}{2}") # `\f` wird missinterpretiert

rac{\pi}{2}


`end backslash1`

`begin backslash2`

In [7]:
len("\\n"), len("\\f"), len("\\p")

(2, 2, 2)

In [8]:
print("\\frac{\\pi}{2}")

\frac{\pi}{2}


`end backslash2`

`begin backslash3`

In [9]:
len(r"\n"), len(r"\f"), len(r"\p")

(2, 2, 2)

In [10]:
print(r"\frac{\pi}{2}")

\frac{\pi}{2}


`end backslash3`

### Scheinbar wirkungsloser Modul-Import

In [11]:
# leeres Modul anlegen
!touch human_challenges_solved.py

`begin reload1`

In [12]:
import human_challenges_solved

# Inzwischen gab es Änderungen am Code ... -> Neu laden!

import importlib
importlib.reload(human_challenges_solved)
pass #!

`end reload1`

### Verwirrung mit lambdify und Arrays

`begin lambdify1`

In [13]:
import sympy as sp
import numpy as np

a, x = sp.symbols("a, x")
# a-abhängige Matrix einsetzen:
M = sp.Matrix([x, a*x**2])
# unproblematischen Wert für a einsetzen
M1 = M.subs(a, 1)
f1 = sp.lambdify(x, M1)
res1 = f1(np.array([1, 2, 3]))

array([[[1, 2, 3]],

       [[1, 4, 9]]])

`end lambdify1`

In [14]:
res = """\
array([[array([1, 2, 3])],
       [0]], dtype=object)
"""
pass

`begin lambdify2`

In [15]:
# problematischen Wert für a einsetzen
M0 = M.subs(a, 0)
f0 = sp.lambdify(x, M0)
# vermeide DeprecationWarning in Ausgabe -> nicht ausführen sondern nur Ergebnis anzeigen #!
#!res0 = f0(np.array([1, 2, 3]))
print(res) #!

array([[array([1, 2, 3])],
       [0]], dtype=object)



`end lambdify2`

In [16]:
# Zur Demo: obigen Code ausführen (erzeugt mit aktuellem sympy (1.8) und numpy (1.20.1) eine DeprecationWarning)
res0 = f0(np.array([1, 2, 3]))

<lambdifygenerated-2>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return (array([[x], [0]]))


array([[array([1, 2, 3])],
       [0]], dtype=object)

`begin lambdify3`

In [17]:
# "Nahrhafte" Null einfügen (x - x)
M0[1] = sp.Add(M0[1], x, - x, evaluate=False)

In [18]:
M0

Matrix([
[         x],
[-x + x + 0]])

In [19]:
f0 = sp.lambdify(x, M0)
res0 = f0(np.array([1, 2, 3]))

array([[[1, 2, 3]],

       [[0, 0, 0]]])

`end lambdify3`

`begin lambdify4`

In [20]:
res0.shape, res1.shape

((2, 1, 3), (2, 1, 3))

`end lambdify4`

`begin lambdify5`

In [21]:
res0.squeeze()

array([[1, 2, 3],
       [0, 0, 0]])

`end lambdify5`

### Rückgabetyp von sympy.solve

`begin solve1`

In [22]:
import sympy as sp
x, y = sp.symbols("x, y")

# Skalare Gleichung mit 2 Lösungen -> Liste
sp.solve(x**2-4)

[-2, 2]

In [23]:

# Gleichungssystem mit 1 Lösung -> dict
sp.solve([x-4, y-9])

{x: 4, y: 9}

In [24]:

# Gleichungssystem mit 2 Lösungen -> Liste mit dicts
sp.solve([x**2-4, y**2-9])

[{x: -2, y: -3}, {x: -2, y: 3}, {x: 2, y: -3}, {x: 2, y: 3}]

`end solve1`

`begin solve2`

In [25]:
sp.solve(x**2-4, dict=True)

[{x: -2}, {x: 2}]

In [26]:

sp.solve([x-4, y-9], dict=True)

[{x: 4, y: 9}]

`end solve2`

In [27]:
# original Fehlermeldung: 
err_msg = "TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''"
# fürs Buch gekürzt: 
err_msg = "TypeError: ufunc 'isfinite' not supported for input types ..."

"TypeError: ufunc 'isfinite' not supported for input types ..."

`begin numpification1`

In [28]:
M = sp.Matrix([[0, 2], [3, 4]])/2

Matrix([
[  0, 1],
[3/2, 2]])

In [29]:
M_num = np.array(M)

array([[0, 1],
       [3/2, 2]], dtype=object)

In [30]:
#!np.linalg.eigvals(M_num) # Eigenwertberechnung schlägt fehl
print(f"{err_msg}") #!

TypeError: ufunc 'isfinite' not supported for input types ...


`end numpification1`

`begin numpification2`

In [31]:
# type-Funktion für elementweise Anwendung vorbereiten ...
vtype = np.vectorize(type)
vtype(M_num) # ... und anwenden

array([[<class 'sympy.core.numbers.Zero'>,
        <class 'sympy.core.numbers.One'>],
       [<class 'sympy.core.numbers.Rational'>,
        <class 'sympy.core.numbers.Integer'>]], dtype=object)

`end numpification2`

`begin numpification3`

In [32]:
M_num2 = np.array(M, dtype=float)

array([[0. , 1. ],
       [1.5, 2. ]])

In [33]:
np.linalg.eigvals(M_num2) # Eigenwertberechnung funktioniert

array([-0.58113883,  2.58113883])

`end numpification3`

### Versehentliches Broadcasting

`begin broadcasting1`

In [34]:
x = np.arange(3)+100

array([100, 101, 102])

In [35]:
y = np.arange(3).reshape(1, -1) # 1 Zeile, passende Spaltenzahl

array([[0, 1, 2]])

In [36]:
# 1D-Array + Zeile = Zeile -> vermutlich erwartet
x + y

array([[100, 102, 104]])

`end broadcasting1`

`begin broadcasting2`

In [37]:
# 1D-Array + Spalte = Matrix -> vermutlich nicht beabsichtigt
x + y.T

array([[100, 101, 102],
       [101, 102, 103],
       [102, 103, 104]])

`end broadcasting2`

## »Schwarze Python Magie« 

Code der auf den ersten Blick verwirrend oder falsch aussieht und überraschende Ergebnisse liefert.

### Indextricksereien

`begin indextricks1`

In [38]:
from numpy import r_, c_

In [39]:
r_[0:20:3] # Abk. für np.arange(start, stop, step)

array([ 0,  3,  6,  9, 12, 15, 18])

In [40]:
r_[0:12:7j] # Abk. für np.linspace(start, stop, N)

array([ 0.,  2.,  4.,  6.,  8., 10., 12.])

`end indextricks1`

`begin indextricks2`

In [41]:
x = r_[3, 4, 5, 100] # Abk. für np.array(...)

array([  3,   4,   5, 100])

In [42]:
y = r_[0, x, [7, 8], 1:4 ] # Einfaches Zusammenfügen

array([  0,   3,   4,   5, 100,   7,   8,   1,   2,   3])

`end indextricks2`

`begin indextricks3`

In [43]:
c_[x, 3*x, range(4), 10:14, [21, 22, 23, 24]]

array([[  3,   9,   0,  10,  21],
       [  4,  12,   1,  11,  22],
       [  5,  15,   2,  12,  23],
       [100, 300,   3,  13,  24]])

`end indextricks3`

`begin indextricks4`

In [44]:
type(r_) # RClass steht für Row-Class, CClass analog

numpy.lib.index_tricks.RClass

`end indextricks4`

### Ungewöhnliche Nutzung von Fallunterscheidungen (in Zuweisung und als Index)

`begin booltricks1`

In [45]:
for i in range(3):
    x = i if i < 2 else "zu groß"
    print(x)

0
1
zu groß


`end booltricks1`

`begin booltricks2`

In [46]:
data = None # Zum Beispiel aus Funktionsaufruf
msg = data or "Keine Daten erhalten!"
print(msg)

Keine Daten erhalten!


In [47]:
for i in range(3):
    msg = ("klein", "groß")[i > 1]
    print(msg)

klein
klein
groß


`end booltricks2`

### »Spezielle«  Module

In [48]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


`begin special_modules1`

In [49]:
#!import this
# importlib.reload(this) #!
# Standardmäßig keine Ausgabe, um Pointe nicht zu verderben #!

`end special_modules1`

Der folgende Aufruf öffnet den Browser (Blick in den Quellcode des Moduls entzaubert die Magie)

`begin special_modules2`

In [50]:
import antigravity

`end special_modules2`

`begin special_modules3`

In [51]:
import os
import sys
os.path.abspath(this.__file__)

'/home/ck2/miniconda3/envs/base38/lib/python3.8/this.py'

In [52]:
os.path.abspath(antigravity.__file__)

'/home/ck2/miniconda3/envs/base38/lib/python3.8/antigravity.py'

`end special_modules3`